In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try:
    import google.colab
    colab = True
except:
    colab = False

In [3]:
if colab is True:
    # Running in Google Colab
    # Clone the repo
    !git clone https://github.com/sicara/easy-few-shot-learning
    %cd easy-few-shot-learning
    !pip install .
else:
    # Run locally
    # Ensure working directory is the project's root
    # Make sure easyfsl is installed!
    %cd ..

Cloning into 'easy-few-shot-learning'...
remote: Enumerating objects: 1188, done.
remote: Counting objects: 100% (451/451), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 1188 (delta 285), reused 259 (delta 204), pack-reused 737
Receiving objects: 100% (1188/1188), 2.33 MiB | 23.40 MiB/s, done.
Resolving deltas: 100% (689/689), done.
/content/easy-few-shot-learning
Processing /content/easy-few-shot-learning
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 

In [5]:
!unzip /content/BasicFinalDatabase_FSL.zip -d /content/data

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000060.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000061.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000062.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000063.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000064.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000065.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000066.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000067.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000068.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000069.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000070.bmp  
  inflating: /content/data/BasicFinalDatabase_FSL/Train/201/bcc000071.bmp  
  inflating: /content/data/BasicFinal

In [6]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import torchvision
import torch.utils.data

In [7]:
random_seed = 30
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Episodic Training

In [8]:
n_way = 5
n_shot = 1
n_query = 10

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_workers = 0

In [9]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class BengaliCharactersDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = []

        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            for img_file in os.listdir(class_dir):
                if img_file.endswith('.bmp'):
                    self.samples.append((os.path.join(class_dir, img_file), class_name))

        # Debug: Print the first few samples
        print("First few samples:")
        print(self.samples[:5])  # Adjust the number to print more or fewer samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label_str = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        # Convert label from string to integer
        label = self.class_to_idx[label_str]

        if self.transform:
            image = self.transform(image)

        return image, label

    def get_labels(self):
        return [label for _, label in self.samples]


In [10]:
image_size = 84  # Adjusted for ResNet, which typically expects larger input sizes

train_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

train_set = BengaliCharactersDataset(root_dir='/content/data/BasicFinalDatabase_FSL/Train', transform=train_transforms)
test_set = BengaliCharactersDataset(root_dir='/content/data/BasicFinalDatabase_FSL/Test', transform=test_transforms)


First few samples:
[('/content/data/BasicFinalDatabase_FSL/Train/195/bcc000118.bmp', '195'), ('/content/data/BasicFinalDatabase_FSL/Train/195/bcc000133.bmp', '195'), ('/content/data/BasicFinalDatabase_FSL/Train/195/bcc000131.bmp', '195'), ('/content/data/BasicFinalDatabase_FSL/Train/195/bcc000180.bmp', '195'), ('/content/data/BasicFinalDatabase_FSL/Train/195/bcc000058.bmp', '195')]
First few samples:
[('/content/data/BasicFinalDatabase_FSL/Test/179/bcc000118.bmp', '179'), ('/content/data/BasicFinalDatabase_FSL/Test/179/bcc000133.bmp', '179'), ('/content/data/BasicFinalDatabase_FSL/Test/179/bcc000131.bmp', '179'), ('/content/data/BasicFinalDatabase_FSL/Test/179/bcc000180.bmp', '179'), ('/content/data/BasicFinalDatabase_FSL/Test/179/bcc000058.bmp', '179')]


In [11]:
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


n_tasks_per_epoch = 500
n_validation_tasks = 100

# Directly use the test set for validation
val_set = test_set

train_set.get_labels = lambda: [
    instance[1] for instance in train_set
]

val_set.get_labels = lambda: [
    instance[1] for instance in val_set
]

# Those are special batch samplers that sample few-shot classification tasks with a pre-defined shape
train_sampler = TaskSampler(
    train_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch
)
val_sampler = TaskSampler(
    val_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)

# Finally, the DataLoader. We customize the collate_fn so that batches are delivered
# in the shape: (support_images, support_labels, query_images, query_labels, class_ids)
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

print(len(train_set), len(val_set))

9360 2640


In [21]:
#Only used this encoder for RelationNetworks few shot algorithm

class CNNEncoder(nn.Module):
    """docstring for ClassName"""
    def __init__(self):
        super(CNNEncoder, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(3,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=0),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU(),
                        nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())
        self.layer4 = nn.Sequential(
                        nn.Conv2d(64,64,kernel_size=3,padding=1),
                        nn.BatchNorm2d(64, momentum=1, affine=True),
                        nn.ReLU())

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        #out = out.view(out.size(0),-1)
        return out # 64

from easyfsl.methods import RelationNetworks, FewShotClassifier

convolutional_network = CNNEncoder()
few_shot_classifier = RelationNetworks(convolutional_network,feature_dimension = 64).to(DEVICE)

In [22]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 30
scheduler_milestones = [120, 160]
scheduler_gamma = 0.1
learning_rate = 1e-2
tb_logs_dir = Path(".")

train_optimizer = SGD(
    few_shot_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [23]:
def training_epoch(
    model: FewShotClassifier, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.to(DEVICE), support_labels.to(DEVICE)
            )
            classification_scores = model(query_images.to(DEVICE))

            loss = LOSS_FUNCTION(classification_scores, query_labels.to(DEVICE))

            loss.backward()

            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [24]:
from easyfsl.utils import evaluate


best_state = few_shot_classifier.state_dict()
best_validation_accuracy = 0.0
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = few_shot_classifier.state_dict()
        print("Ding ding ding! We found a new best model!")

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

    if epoch % 1 == 0 : torch.save(best_state, '/content/drive/MyDrive/PatternRecognition-main/ML/Proto/real_1shot_5way_BanglaPrototypicalNetworks_BanglaLekha_Final.pth')

Epoch 0


Validation: 100%|██████████| 100/100 [00:04<00:00, 20.69it/s, accuracy=0.614]


Ding ding ding! We found a new best model!
Epoch 1


Validation: 100%|██████████| 100/100 [00:04<00:00, 21.96it/s, accuracy=0.65]


Ding ding ding! We found a new best model!
Epoch 2


...

Validation: 100%|██████████| 100/100 [00:04<00:00, 20.38it/s, accuracy=0.774]


Epoch 28


Validation: 100%|██████████| 100/100 [00:04<00:00, 21.73it/s, accuracy=0.756]


Epoch 29


Validation: 100%|██████████| 100/100 [00:04<00:00, 21.70it/s, accuracy=0.771]


In [ ]:
# PATH = '/content/drive/MyDrive/PatternRecognition-main/ML/Proto/5shot_5way_BanglaPrototypicalNetworks_BanglaLekha_Isolated.pth'
# few_shot_classifier.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))


In [25]:
PATH = '/content/drive/MyDrive/PatternRecognition-main/ML/Proto/real_1shot_5way_BanglaPrototypicalNetworks_BanglaLekha_Final.pth'
few_shot_classifier.load_state_dict(torch.load(PATH))

<All keys matched successfully>

**Evaluation**

In [26]:
n_test_tasks = 100

test_set.get_labels = lambda: [
    instance[1] for instance in test_set
]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)
test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [34]:
from easyfsl.utils import evaluate
accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|██████████| 100/100 [00:04<00:00, 22.02it/s, accuracy=0.775]

Average accuracy : 77.54 %


In [28]:
from sklearn.metrics import precision_recall_fscore_support
import torch


def evaluate_with_metrics(model, loader, device):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for _, (support_images, support_labels, query_images, query_labels, _) in enumerate(loader):
            model.process_support_set(support_images.to(device), support_labels.to(device))
            outputs = model(query_images.to(device))
            _, preds = torch.max(outputs, 1)
            all_labels.extend(query_labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    precision, recall, f1_score, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
    return precision, recall, f1_score


In [29]:
precision, recall, f1_score = evaluate_with_metrics(few_shot_classifier, test_loader, DEVICE)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

Precision: 0.76
Recall: 0.76
F1 Score: 0.76


In [ ]:
def predict_image_with_support_set(query_image_path):
    # Load and transform the query image
    query_image = Image.open(query_image_path).convert('RGB')
    query_image_tensor = image_transforms(query_image).unsqueeze(0)  # Add batch dimension

    # Set the model in evaluation mode
    few_shot_classifier.eval()

    # Use the model to predict
    with torch.no_grad():
        few_shot_classifier.process_support_set(support_images_tensor.to(DEVICE), support_labels_tensor.to(DEVICE))
        outputs = few_shot_classifier(query_image_tensor.to(DEVICE))

    probabilities = torch.softmax(outputs, dim=1)
    predicted_label_idx = probabilities.argmax(1).item()
    predicted_label = support_labels[predicted_label_idx] + 1  # Adjust label to match original range

    return predicted_label



import torch
from torchvision import transforms
from PIL import Image
import os


label_to_bengali = {
    1: "অ",
    2: "আ",
    3: "ই",
    4: "ঈ",
    5: "উ",
    6: "ঊ",
    7: "ঋ",
    8: "এ",
    9: "ঐ",
    10: "ও",
    11: "ঔ",
    12: "ক",
    13: "খ",
    14: "গ",
    15: "ঘ",
    16: "ঙ",
    17: "চ",
    18: "ছ",
    19: "জ",
    20: "ঝ",
    21: "ঞ",
    22: "ট",
    23: "ঠ",
    24: "ড",
    25: "ঢ",
    26: "ণ",
    27: "ত",
    28: "থ",
    29: "দ",
    30: "ধ",
    31: "ন",
    32: "প",
    33: "ফ",
    34: "ব",
    35: "ভ",
    36: "ম",
    37: "য",
    38: "র",
    39: "ল",
    40: "শ",
    41: "ষ",
    42: "স",
    43: "হ",
    44: "ড়",
    45: "ঢ়",
    46: "য়",
    47: "ৎ",
    48: "ং",
    49: "ঃ",
    50: "ঁ",
    51: "০",
    52: "১",
    53: "২",
    54: "৩",
    55: "৪",
    56: "৫",
    57: "৬",
    58: "৭",
    59: "৮",
    60: "৯",
}


# Function to print the Bengali character for a given label
def print_bengali_character(label):
    character = label_to_bengali.get(label, "Unknown label")
    print(character)


# Define the image preprocessing
image_transforms = transforms.Compose([
    transforms.Resize((84, 84)),  # Resize the image to what the model expects
    transforms.ToTensor(),        # Convert the image to a tensor
])

# Define the root directory of your support images
root_dir = '/content/data/BanglaLekha_Isolated_mod/Test/'

# Generate support set
support_images = []
support_labels = []

# Assuming each class directory contains images for that class
for label in range(1, 12):  # Labels from 1 to 60
    class_dir = os.path.join(root_dir, str(label))
    if not os.path.exists(class_dir):
        continue  # Skip if the class directory doesn't exist

    # List all images in the class directory
    img_files = [f for f in os.listdir(class_dir) if f.endswith('.png')]
    for img_file in img_files[:1]:  # Take only the first image for simplicity; adjust as needed
        img_path = os.path.join(class_dir, img_file)
        image = Image.open(img_path).convert('RGB')
        support_images.append(image_transforms(image))
        support_labels.append(label - 1)  # Adjust label to start from 0

# Convert lists to tensors
support_images_tensor = torch.stack(support_images)
support_labels_tensor = torch.tensor(support_labels)

query_image_path = '/content/mota_dho.jpg'
predicted_label = predict_image_with_support_set(query_image_path)
print("Predicted label: ")
print_bengali_character(predicted_label)

Predicted label: 
ষ
